# **Compliance GPT with LangChain**

## **Setup**

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [3]:
from utils.config import get_config
from utils.models import ModelName, get_model

config = get_config()

In [4]:
USER_ID = 'xmriz'
CONVERSATION_ID = 'xmriz-2021-07-01-01'

## **Define Model**

In [5]:
model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config)

## **Load Vector Store Index**

In [6]:
from databases.vector_store import RedisIndexManager

chat_store_ojk = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
chat_store_bi = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)

vector_store_ojk = chat_store_ojk.load_vector_index()
vector_store_bi = chat_store_bi.load_vector_index()

c:\Users\acer\miniconda3\envs\chatbot-ocbc\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## **Get Retriever**

In [6]:
from retriever.retriever_bi_ojk.retriever_bi_ojk import get_retriever_bi_ojk, get_combined_retriever_bi_ojk
from langchain.chains.query_constructor.base import AttributeInfo


# metadata_field_info_bi = [
#     AttributeInfo(
#         name="title",
#         description="The title of the document of regulation",
#         type="string",
#     ),
#     AttributeInfo(
#         name="date",
#         description="The date of the document of regulation",
#         type="string",
#     ),
#     AttributeInfo(
#         name="type_of_regulation",
#         description="The type of the regulation",
#         type="string",
#     ),
#     AttributeInfo(
#         name="sector",
#         description="The sector of the regulation",
#         type="string",
#     ),
# ]

# retriever_bi = get_retriever_bi_ojk(
#     vector_store=vector_store_bi,
#     embed_model=embed_model,
#     llm_model=llm_model,
#     config=config,
#     top_k=20,
#     top_n=6,
#     metadata_field_info=metadata_field_info_bi,
# )


metadata_field_info_ojk = [
    AttributeInfo(
        name="title",
        description="The title of the document of regulation",
        type="string",
    ),
    AttributeInfo(
        name="sector",
        description="The sector of the regulation",
        type="string",
    ),
    AttributeInfo(
        name="subsector",
        description="The subsector of the regulation",
        type="string",
    ),
    AttributeInfo(
        name="regulation_type",
        description="The type of the regulation",
        type="string",
    ),
    AttributeInfo(
        name="regulation_number",
        description="The number of the regulation",
        type="string",
    ),
    AttributeInfo(
        name="effective_date",
        description="The effective date of the regulation",
        type="string",
    ),
]

retriever_ojk = get_retriever_bi_ojk(
    vector_store=vector_store_ojk,
    embed_model=embed_model,
    llm_model=llm_model,
    config=config,
    top_k=20,
    top_n=6,
)

# retriever = get_combined_retriever_bi_ojk(
#     vector_store_ojk=vector_store_ojk,
#     vector_store_bi=vector_store_bi,
#     embed_model=embed_model,
#     llm_model=llm_model,
#     config=config,
#     top_k=20,
#     top_n=6,
# )

In [7]:
retriever_ojk.invoke("Berapa lama waktu maksimum dalam hari kerja yang diberikan kepada Bank Indonesia untuk memberitahukan keputusan kepada Lembaga Sertifikasi Profesi tentang permohonannya sebagaimana dimaksud dalam Pasal 21??")

[Document(metadata={'id': 'doc:ojk:4effeb544d3c4a2da39e713b48c8b8fc', 'title': 'Peraturan Bank Indonesia tentang Produk Bank Syariah dan Unit Usaha Syariah', 'sector': 'Perbankan,  Syariah', 'subsector': 'Bank Umum,  Perbankan Syariah', 'regulation_type': 'PPBI', 'regulation_number': '10/17/PBI/2008', 'effective_date': '28 Desember 2016', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/PBI-tentang-Produk-Bank-Syariah-dan-Unit-Usaha-Syariah/169.pdf', 'doc_id': '1070', 'page_number': '4', 'relevance_score': 0.10894505}, page_content='dimaksud pada ayat (1) paling lambat 15 (lima belas) hari sejak \nseluruh persyaratan dipenuhi dan dokumen pelaporan diterima \nsecara lengkap. \n(3) Bank dilarang mengeluarkan Produk baru dalam jangka waktu 15 \n(lima belas) hari sebagaimana dimaksud pada ayat (2), apabila \nbelum memperoleh penegasan tidak keberatan dari Bank \nIndonesia. \n(4) Apabila dalam jangka waktu 15 (lima belas) hari setelah seluruh \npersyaratan dipenuhi dan dokumen

## **Create Chain**

In [13]:
from constant.ojk.prompt import CONTEXTUALIZE_Q_PROMPT_STR, QA_SYSTEM_PROMPT_STR
from databases.chat_store import RedisChatStore
from chain.rag_chain import create_chain, create_chain_with_chat_history

chat_store = RedisChatStore(k=5, config=config, db_id=1)

chain = create_chain(
    contextualize_q_prompt_str=CONTEXTUALIZE_Q_PROMPT_STR,
    qa_system_prompt_str=QA_SYSTEM_PROMPT_STR,
    retriever=retriever_ojk,
    llm_model=llm_model,
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store,
)

## **Invoke**

#### **With Question, Answer, and Context**

In [12]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="What is the minimum term length for a Bank Business Plan?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

{'rewrited question': 'Berapa lama minimal jangka waktu untuk Rencana Bisnis Bank?',
 'answer': 'Tidak ada informasi yang relevan dalam konteks yang diberikan mengenai jangka waktu minimal untuk Rencana Bisnis Bank.',
 'context': '[\n  {\n    "metadata": {\n      "id": "doc:bi:a4cf4d51606f4a8e8d95b77327ee9068",\n      "file_id": "33a9232e",\n      "title": "SURAT EDARAN BANK INDONESIA NO.13/21/DSM TANGGAL 15 AGUSTUS 2011 PERIHAL PELAPORAN KEGIATAN LALU LINTAS DEVISA LEMBAGA BUKAN BANK",\n      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/7c3f7d4a743a483fa37cdbfe0bbd8134lamp_SE_132112.zip",\n      "date": "15 Agustus 2011",\n      "type_of_regulation": "Surat Edaran",\n      "sector": "Moneter",\n      "standardized_extracted_file_name": "sebi-13_21_dsm-15082011-surat_edaran_bank_indonesia_no13_21_dsm_tanggal_15_agustus_2011_perihal_pela-lamp_buku_pedoman",\n      "standardized_file_name": "sebi-13_21_dsm-15082011-surat_edaran_bank_indonesia_no13_21_dsm_tanggal_15

In [11]:
print(response['context'])

[
  {
    "metadata": {
      "id": "doc:bi:9f525c3e76e44bb7bc1751f692dd6fa1",
      "file_id": "2de48a27",
      "title": "SURAT EDARAN BANK INDONESIA NO.15/5/DSM TANGGAL 7 MARET 2013 PERIHAL PELAPORAN KEGIATAN LALU LINTAS DEVISA SELAIN UTANG LUAR NEGERI",
      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/f16e4f7524784cf188b873895a89c833lampiran_se_150514.zip",
      "date": "7 Maret 2013",
      "type_of_regulation": "Surat Edaran",
      "sector": "Moneter",
      "standardized_extracted_file_name": "sebi-15_5_dsm-07032013-surat_edaran_bank_indonesia_no15_5_dsm_tanggal_7_maret_2013_perihal_pelaporan-lampiran_3_final",
      "standardized_file_name": "sebi-15_5_dsm-07032013-surat_edaran_bank_indonesia_no15_5_dsm_tanggal_7_maret_2013_perihal_pelaporan_kegiatan_lalu_lintas_devisa_selain_utang_luar_negeri",
      "relevance_score": 0.0015669888
    },
    "page_content": "Korea sebesar EUR500,000 Perusahaan R Ltd. tidak mendapatkan bunga dari simpanan pada France

#### **Streaming**

In [10]:
# from chain.chain_ojk.chain_ojk import print_answer_stream

# print_answer_stream(chain=chain, question="Siapa itu lionel messi?", user_id=USER_ID, conversation_id=CONVERSATION_ID)

Saya minta maaf, tetapi pertanyaan Anda tidak terkait dengan konteks yang diberikan. Konteks yang diberikan adalah tentang kepatuhan perbankan di Indonesia. Apakah ada pertanyaan lain yang dapat saya bantu jawab?

#### **Chat Store Monitor**

In [10]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: Berapa limit transaksi dari QRIS?
AI: Limit transaksi dari QRIS adalah paling banyak Rp5.000.000,00 (lima juta rupiah) pada setiap rekening. 

Source: Surat Edaran OJK 34/SEOJK.03/2021 (https://www.ojk.go.id/id/regulasi/Documents/Pages/Buku-Panduan-Akuntansi-Perbankan-Bagi-Bank-Umum-Konvensional-/SEOJK%2034%20-%2003%20-%202021.pdf#page=736)


In [17]:
# to make the chat history empty
chat_store.clear_all()

In [9]:
import redis

r = redis.Redis(host='108.137.122.156', port=6379, db=0)

# Variabel untuk menghitung jumlah kunci yang memenuhi kriteria
matching_keys_count = 0

# Gunakan SCAN untuk mengiterasi semua kunci
cursor = '0'
while cursor != 0:
    cursor, keys = r.scan(cursor=cursor, match='*', count=1000)
    for key in keys:
        if r.hexists(key, 'doc_id'):
            doc_id = r.hget(key, 'doc_id')
            try:
                if int(doc_id) > 1111:
                    matching_keys_count += 1
            except ValueError:
                continue

print(f"Jumlah kunci dengan doc_id > 1111: {matching_keys_count}")

KeyboardInterrupt: 